# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [110]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [111]:
def scrape_imdb(title_type,user_rating,start_date,end_date):
    title_type_clean = title_type.lower().strip().replace(' ','_')
    date_time_start = datetime.strptime(start_date,'%B %d, %Y')
    string_start = datetime.strftime(date_time_start,"%Y-%m-%d")
    date_time_end = datetime.strptime(end_date,'%B %d, %Y')
    string_end = datetime.strftime(date_time_end,"%Y-%m-%d")
    url=f'https://www.imdb.com/search/title/?title_type={title_type_clean}&user_rating={user_rating},10&release_date={string_start},{string_end}'
    
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.content)
    
    
    Title_position = soup.find_all('h3')[:-1]
    Position = [t.text.split('.')[0] for t in Title_position]
    Title = [t.text.split('. ',1)[1] for t in Title_position]
    Years = [div.find('span').text for div in soup.find_all('div',attrs={'class':'sc-43986a27-7 dBkaPT dli-title-metadata'})]
    Title_urls = ['https://www.imdb.com' + u.get('href') for u in soup.select('a.ipc-title-link-wrapper')]
    Votes = [div.text.replace('Votes','').replace(',','') for div in soup.find_all('div', class_="sc-53c98e73-0 kRnqtn")]

    Runtime = []
    for div in soup.find_all('div',attrs={'class':"sc-43986a27-7 dBkaPT dli-title-metadata"}):
        try:
            Runtime.append(div.find_all('span')[1].text)
        except:
            Runtime.append(None)
            
    Description = []
    for l in soup.find_all('li',attrs={'class':"ipc-metadata-list-summary-item"}):
        try:
            Description.append(l.find('div', attrs={'class':"ipc-html-content-inner-div"}).text)
        except:
            Description.append(None)

    Stars = [stars.get('aria-label').split(': ')[1] for stars in soup.find_all('span',attrs ={'class':"ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating"})]
        
    df = pd.DataFrame({'Position':Position,'Title':Title,'Year of Release':Years,
             'Votes':Votes,'Runtime':Runtime,'Decription':Description,
              'Stars':Stars,'URL':Title_urls})
    return df

In [112]:
title_type='Short'
user_rating = 7.5
start_date='January 1, 1990'
end_date ='December 31, 1992'

scrape_imdb(title_type, user_rating, start_date, end_date)

,Position,Title,Year of Release,Votes,Runtime,Decription,Stars,URL
0,1,Mr. Bean Takes an Exam,1991,85,9m,Mr. Bean realizes he doesn't know the answers ...,8.2,https://www.imdb.com/title/tt16358846/?ref_=sr...
1,2,Knock Out,1992,7,19m,None,8.1,https://www.imdb.com/title/tt0104628/?ref_=sr_t_2
2,3,Anima Mundi,1991,1331,28m,Image and music are intertwined in this third ...,7.5,https://www.imdb.com/title/tt0101349/?ref_=sr_t_3
3,4,Härlig är jorden,1991,2478,14m,After witnessing an act of unprecedented viole...,7.5,https://www.imdb.com/title/tt0102083/?ref_=sr_t_4
4,5,12:01 PM,1990,1481,25m,Poor Myron Castleman is stuck reliving the sam...,7.6,https://www.imdb.com/title/tt0098962/?ref_=sr_t_5
5,6,Jacob's Ladder,1990,57,38m,Jake has withdrawn and lives alone. Busking is...,7.7,https://www.imdb.com/title/tt0884859/?ref_=sr_t_6
6,7,Jídlo,1992,3522,17m,Examines the human relationship with food by s...,8.0,https://www.imdb.com/title/tt0104285/?ref_=sr_t_7
7,8,Purple Triangles,1991,141,25m,A documentary about Jehovah's Witnesses in Naz...,8.3,https://www.imdb.com/title/tt0162577/?ref_=sr_t_8
8,9,Big House Blues,1990,231,7m,"Ren and Stimpy, on the streets starving, are c...",8.0,https://www.imdb.com/title/tt0448204/?ref_=sr_t_9
9,10,Sadako and the Thousand Paper Cranes,1991,27,30m,Sadako was two years old when the A-bomb explo...,7.6,https://www.imdb.com/title/tt0102827/?ref_=sr_...


## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [113]:
# Your solution goes here